# Improving the Search Index

**IMPORTANT: This is not the final version! The assignments will be changed.**

In this mini-assignment, we will improve the search index and query functions from the previous mini-assignment.

## Loading the Data and Defining Auxiliary Functions

This section is copied from the previous notebook.

In [1]:
import pickle, bz2, re
from collections import namedtuple, defaultdict, Counter
from IPython.display import display, HTML
from math import log10

Summaries_file = 'data/malaria__Summaries.pkl.bz2'
Abstracts_file = 'data/malaria__Abstracts.pkl.bz2'

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )
for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )

import pickle, bz2, re
from collections import namedtuple, defaultdict, Counter
from IPython.display import display, HTML
from math import log10

Summaries_file = 'data/malaria__Summaries.pkl.bz2'
Abstracts_file = 'data/malaria__Abstracts.pkl.bz2'

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )
for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )

In [2]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

In [3]:
def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/%s>%s</a>' % (s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>%s</em></small>' % Abstracts[id])
    if (show_id):
        lines.append('[ID: %d]' % id)
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

In [4]:
inverted_index = defaultdict(set)

for (id, abstract) in Abstracts.items():
    for term in preprocess(tokenize(abstract)):
        inverted_index[term].add(id)

## Stemming

As we could see from the results of the last assignment, our simple index doesn't handle punctuation and the difference between singular and plural versions of the same word very well. Fortunately, Python's _NLTK_ package provides implementations of these algorithms we can use. If your Python package doesn't already come with it, you have to install _NLTK_ by following [these instructions](http://www.nltk.org/install.html).

In [6]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import EnglishStemmer
import nltk
nltk.download('punkt')
stemmer = EnglishStemmer()

s = '''Good muffins cost $3.88\nin New York.  Please buy me two of them.\n\nThanks.'''

print(tokenize(s))
print(word_tokenize(s))

['Good', 'muffins', 'cost', '$3.88\nin', 'New', 'York.', '', 'Please', 'buy', 'me', 'two', 'of', 'them.\n\nThanks.']
['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.', 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']


In [7]:
print(stemmer.stem("processes"))

process


## Ranking

Another way to improve our search results is to *rank* them. A possible way to do this is to calculate a score for each document based on the matching terms from the query. One such scoring method is *tf-idf*, as explained in the lecture slides.

In order to quickly calculate the scores for a term/document combination, we'll need quick access to a couple of things:

- tf(t,d): How often does a term occur in a document
- df(t): In how many documents does a term occur
- N: The number of documents in our index

In [8]:
tf_matrix = defaultdict(Counter)
for (id, abstract) in Abstracts.items():
    tf_matrix[id] = Counter(preprocess(tokenize(abstract)))

def tf(t,d):
    return float(tf_matrix[d][t])

def df(t):
    return float(len(inverted_index[t]))
    
def num_documents():
    return float(len(Abstracts))

Let's test these functions with some examples:

In [9]:
print(tf('network', 24130474))
print(df('network'))
print(num_documents())

3.0
403.0
67028.0


Using these three helper functions, we can now easily calculate the _tf-idf_ weights of a term in a document by implementing the weighting formula from the slides, which you will do in the assignments below.

----------

## Assignments

**Your name:** ...

### Task 1

Implement in the code block below the `smarter_tokenize` function using NLTK's function for tokenization, and the `smarter_preprocess` function to perform stemming in addition to case normalization. Then implement the `smarter_and_query` function based on the two functions above. You can start from the code for `and_query` from the last assignment.

Run the query "red blood cell" with the new `smarter_and_query` function. Does it return our exemplary paper *24130474*? Why or why not?

For practical purposes, the code below generates the smarter index on a subset of the data, as generating an index with stemming on the entire set would take too much time.

In [36]:
# ----------
# Implement these functions according to the task above:

#def smarter_tokenize(text):
#    ...

#def smarter_preprocess(tokens):
#    ...

#def smarter_and_query(query_string):
#    ...

# ----------

# Below, we create our smarter index (based on a subset of the documents for demonstration purposes)
smarter_index = defaultdict(set)

# Here we define the subset (somewhat arbitrary):
subset_of_ids = set(Abstracts.keys()).intersection(set(range(24100000,24200000)))
abstracts_subset = ((k, Abstracts[k]) for k in subset_of_ids)

# Uncomment this line to process the whole corpus (might take a long time):
#abstracts_subset = Abstracts.items()

# Building our smarter index:
for (id, abstract) in abstracts_subset:
    for term in smarter_preprocess(smarter_tokenize(abstract)):
        smarter_index[term].add(id)

**Answer:** [Write your answer text here]

### Task 2

Create a function `tfidf(t,d)` that returns the tf-idf score of term `t` in document `d` by using the `tf(t,d)`, `df(t)` and `num_documents()` functions we defined above. The tf-idf formula can be found on the lecture slides. Test your function with the examples shown below.
  
You can use our old index for this task and the tasks below: You do **not** need to include the results from above with the smarter tokenization and preprocessing functions.
  
You can use the `log10(n)` function to calculate the base 10 logarithm.

In [44]:
#def tfidf(t,d):
#    ...

#print(tfidf('network', 24130474))
#print(tfidf('var', 24130474))
#print(tfidf('surface', 24130474))

### Task 3

Create a function `query1(query_string)`, which accepts as input a single query string that could consist of one or more words, and returns or prints a list of (up to) 10 best matching documents, along with their score. Use _tf-idf_ to calculate document scores based on the query, applying the simple sum-score method. The results should be shown in descending order by score.

You can start by copying your `or_query` function from mini-assignment 2, then expand that to rank the results, making use of the `tfidf(t,d)` function you created earlier.

In [ ]:
# Add your code here

### Task 4

Create a second version of the query function function from Task 3, and call it `query2`. This second version should use the cosine method instead of sum-score.

Use the provided `display_summary(id,extra_text)` function to make the output a bit more 'search engine'-like.

In [ ]:
# Add your code here

### Task 5

Come up with a few example queries to run, and include the output here. Do the results make sense? Why (not)?

In [ ]:
# Add your code here

[Write your answer text here]